<a href="https://colab.research.google.com/github/NoraHK3/DataSciProject/blob/main/Sayidaty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import json
from datetime import datetime
import time
import re

class SaudiDishScraper:
    def __init__(self):
        self.base_url = "https://kitchen.sayidaty.net"
        self.saudi_cuisine_url = f"{self.base_url}/recipes/index/cuisine/2419"  # Saudi cuisine category
        self.dishes_data = []

        # كلمات المكونات الأساسية (مع احتمالات الكتابة)
        self.ingredient_keywords = {
            # بروتينات
            'لحم', 'لحمة', 'لحم غنم', 'غنم', 'ضأن', 'خروف', 'مندي',
            'لحم بقر', 'بقر', 'بقري',
            'دجاج', 'فراخ', 'دجاجة',
            'سمك', 'سمكة', 'سموج', 'سي فود', 'بحري',
            'روبيان', 'جمبري', 'قريدس',
            'بيض', 'بيضة', 'بيضات',
            'كبد', 'كبده', 'كبدة',
            'جمل', 'لحم جمل', 'بعير',

            # أساسيات/نشويات
            'رز', 'أرز', 'الرز', 'ألارز', 'رز بسمتي', 'بسمتي',
            'بر', 'حنطة', 'قمح',
            'خبز', 'خبزه', 'خبزة', 'خبز تنور', 'خبز بر',
            'قرصان',
            'جريش', 'جريشه',
            'مرقوق', 'مَرْقُوق',
            'برغل', 'بورغل',
            'كسكس', 'كسكسي',
            'شعيرية', 'شعريه',
            'مكرونة', 'معكرونة', 'مكارونه', 'معكرونه',
            'عجينة', 'عجين', 'عجينه',
            'فريكة', 'فريكه',

            # بقوليات
            'عدس', 'عدسه', 'عدسية',
            'حمص', 'حمص حب', 'حمص مطحون',
            'فول', 'فول مدمس', 'فول حب',
            'بازلاء', 'بسلة', 'بازيلا',
            'لوبيا', 'لوبية',

            # خضروات
            'بطاطس', 'بطاطا', 'بطاطه',
            'طماطم', 'بندورة', 'بندوره',
            'بصل', 'بصلة', 'بصلات',
            'ثوم', 'ثومه',
            'فلفل', 'فلفل رومي', 'فلفل حار', 'شطة', 'شطه',
            'جزر', 'جزره',
            'كوسة', 'كوسه',
            'باذنجان',
            'سبانخ', 'سبناخ',
            'ملوخية', 'ملوخيا',
            'قرع', 'قرعه', 'يقطين',
            'خيار',
            'ملفوف', 'كرنب',
            'خس', 'خص',
            'جرجير',
            'نعناع', 'نعنع',
            'كزبرة', 'كزبره', 'كسبرة',
            'بقدونس', 'بقدونـس',

            # ألبان ومنتجاتها
            'لبن', 'لبن رائب', 'لبن عيران', 'روب',
            'زبادي', 'زبادى',
            'حليب', 'حليب سائل', 'حليب بودرة',
            'قشطة', 'قشطه',
            'جبن', 'جبنة', 'أجبان',
            'سمن', 'سمنة', 'سمنه',
            'زبدة', 'زبد',

            # زيوت ودهون
            'زيت', 'زيت نباتي', 'زيت ذرة', 'زيت دوار الشمس',
            'زيت زيتون',

            # بهارات وتوابل
            'ملح',
            'فلفل', 'فلفل أسود',
            'كمون',
            'كركم',
            'قرفة', 'دارسين',
            'هيل', 'حبهان',
            'زنجبيل',
            'قرنفل',
            'زعفران',
            'ورق غار', 'ورق لورا',
            'بهارات', 'توابل',
            'شطة', 'شطه',

            # إضافات ومكسرات
            'لوز', 'لوزة',
            'جوز', 'عين جمل',
            'فستق', 'فستق حلبي',
            'صنوبر',
            'سمسم',
            'زبيب',
            'تمر', 'تمور', 'عجوة',
            'دبس تمر',
            'عسل', 'عسل أسود',
            'سكر', 'سكّر',

            # سوائل وصلصات
            'خل', 'خَل',
            'ليمون', 'حامض',
            'عصير ليمون',
            'ماء ورد',
            'ماء زهر',
            'مرق', 'مرقة', 'شوربة',
            'صلصة طماطم', 'معجون طماطم', 'طماطم معجون',

            # مكونات حلويات
            'سميد',
            'طحين', 'دقيق',
            'نشا', 'نشا ذرة',
            'ماء', 'مويه',
            'فانيليا', 'فانيلا',
            'كاكاو', 'كاكاو بودرة',
            'حليب مكثف',
            'كريم', 'كريمة', 'كريمه',

            # أكلات وحلويات سعودية
            'كليجة', 'كليجا',
            'معمول', 'معمول تمر',
            'بسبوسة', 'بسبوسه',
            'لقيمات', 'لقمة القاضي', 'عوامة',
            'كنافة', 'كنافه',
            'قطايف', 'قطائف',
            'عصيدة', 'عصيد',
            'بلاليط', 'شعيرية حلوة'
        }

        # كلمات الكميات (وحدات/أوصاف/أرقام)
        self.quantity_words = {
            # وحدات قياس عامة
            'كوب', 'أكواب', 'كاسات', 'كاس', 'كأس', 'كؤوس',
            'ملعقة', 'معلقة', 'ملاعق', 'ملعقة صغيرة', 'ملعقة كبيرة',
            'ملعقة طعام', 'ملعقة شاي', 'ملعقة قهوة', 'كوبان',
            'مل', 'ملل', 'ملليلتر', 'مليلتر', 'مللي', 'ملّي',
            'لتر', 'ليتر', 'لترات', 'صغيره', 'كبيره', 'صغيرة', 'كبيرة',

            # الوزن
            'جرام', 'غرام', 'غ', 'غم',
            'كيلو', 'كيلوجرام', 'كيلوغرام', 'كج', 'كيلو جرام',

            # قطع وأعداد
            'حبة', 'حبات', 'حبه', 'قطعة', 'قطع', 'قطِع',
            'فص', 'فصوص',
            'شريحة', 'شرائح',
            'عود', 'أعواد',
            'ورقة', 'ورق', 'ورقات',
            'قرص', 'أقراص',
            'حزمة', 'حزم', 'ربطة',

            # أواني وعلب
            'علبة', 'علب', 'عبوة', 'عبوات',
            'كيس', 'اكياس', 'رزمة', 'رزمه',
            'برطمان', 'زجاجة', 'زجاجات',

            # أوصاف الكمية
            'قليل', 'قليلاً', 'قليلة', 'بعض', 'بضع',
            'حوالي', 'تقريباً', 'نحو', 'ما يقارب',
            'مقدار', 'مقدار مناسب', 'كمية', 'كمية مناسبة',

            # أجزاء
            'نصف', 'نص', 'نصفه', 'نصه',
            'ربع', 'ربعين', 'ربعه',
            'ثلث', 'ثلثين', 'ثلثه',
            'ثلاثة أرباع', 'ثلاث ارباع', 'تلاتة ارباع',

            # أوصاف الإضافة
            'رشة', 'رشّة', 'رش', 'رشات',
            'ذرة', 'ذرات',
            'قطرة', 'قطرات',
            'ملء', 'مليء',
            'كامل', 'كاملة',
            'كافي', 'كافية',
            'حسب الرغبة', 'حسب الذوق', 'حسب الحاجة',

            # أرقام عربية وإنجليزية (رموز)
            '0','1','2','3','4','5','6','7','8','9',
            '٠','١','٢','٣','٤','٥','٦','٧','٨','٩',

            # أرقام مكتوبة بالعربي
            'واحد','واحدة','إثنان','اثنان','اثنين','إثنين',
            'ثلاث','ثلاثة','ثلاثه',
            'أربع','أربعة','أربعه',
            'خمس','خمسة','خمسه',
            'ست','ستة','سته',
            'سبع','سبعة','سبعه',
            'ثمان','ثمانية','ثمانيه','ثماني',
            'تسع','تسعة','تسعه',
            'عشر','عشرة','عشره',
            'إحدى عشر','إحدعشر','احدى عشر',
            'إثنا عشر','اثني عشر','اثناعشر',
            'عشرون','عشرين','عشرينه',
            'ثلاثون','ثلاثين',
            'أربعون','اربعين',
            'خمسون','خمسين',
            'ستون','ستين',
            'سبعون','سبعين',
            'ثمانون','ثمانين',
            'تسعون','تسعين',
            'مائة','مئه','مية','مائه',

            # أرقام مكتوبة بالإنجليزي
            'one','two','three','four','five','six','seven','eight','nine','ten',
            'eleven','twelve','thirteen','fourteen','fifteen',
            'sixteen','seventeen','eighteen','nineteen',
            'twenty','thirty','forty','fifty','sixty','seventy','eighty','ninety','hundred'
        }

        # كلمات لطريقة التحضير/أفعال يجب حذفها
        self.method_words = {
            'اضف','أضف','نضيف','يضاف','تضاف','اضافي','أضيفي','ضيفي','ضعي','ضع','توضع','يوضع',
            'اقطع','قطّع','قطعي','فرم','افرمي','مفروم','يُفرم','يطحن','طحن','مطحون','اسلق','اسلقي',
            'سلق','مسلوق','حمّر','تحمير','قلّب','تقليب','شوّح','تشويح','اقلي','قلي','مقلي',
            'اشو','اشوي','شوي','مشوي','اسكب','اسكبي','سكب','اسكبوا','قدّم','قدمي','تقديم',
            'ذوّب','اذابة','ذوبان','اخلط','خلط','امزج','مزج','حرك','تحريك','اعجن','عجن','اعصري','عصر',
            'تبّل','تتبيل','منقوع','نقع','انقعي','صفّي','تصفية','اغسل','غسل','منظف','منظفة','نظّف'
        }

        # أوصاف/نعوت ملازمة للمكوّن لا نحتاجها
        self.adjective_words = {
            'طازج','طازجة','مجمد','مجمدة','كامل','كاملة','مقطع','مقطعة','صغير','كبيرة','متوسط',
            'كبير','ناعمة','ناعم','خشن','خشنة','مفروم','مفرومة','مطحون','مطحونة','أوراق','اوراق','أوراقه',
            'حبة كاملة','بدون','مع','بدون بذور','مقشر','مقشرة','مغسول','مغسولة','حار','حارة','بارد','باردة'
        }

    # ====== أدوات مساعدة للنص ======
    def normalize_ar(self, s: str) -> str:
        """إزالة التشكيل/التطويل وتوحيد الألف/الياء/التاء المربوطة"""
        if not s:
            return ''
        s = re.sub(r'[\u064B-\u0652\u0670]', '', s)  # حركات
        s = s.replace('ـ', '')                        # تطويل
        s = re.sub('[إأآا]', 'ا', s)                 # الألف
        s = s.replace('ى', 'ي')                       # ألف مقصورة -> ياء
        s = s.replace('ؤ', 'و').replace('ئ', 'ي')     # همزات
        s = s.replace('ة', 'ه')                       # تاء مربوطة -> ه
        return s.strip()

    def tokenize(self, s: str):
        """تقطيع بسيط على المسافات وعلامات الترقيم"""
        s = re.sub(r'[^\w\u0600-\u06FF]+', ' ', s)
        parts = [p for p in s.split() if p]
        return parts

    # ====== الشبكات/الجلب ======
    def get_page_content(self, url):
        """Fetch page content with error handling"""
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            return None

    def extract_dish_links(self, html_content):
        """Extract all dish links from a listing page"""
        soup = BeautifulSoup(html_content, 'html.parser')
        dish_links = []
        links = soup.select('a[href*="/node/"]')
        for link in links:
            href = link.get('href')
            if href and '/node/' in href:
                full_url = href if href.startswith('http') else f"{self.base_url}{href}"
                dish_links.append(full_url)
        return list(set(dish_links))  # Remove duplicates

    # ====== تنظيف/تحقق من المكونات ======
    def clean_ingredient_text(self, text):
        """احذف الكميات والأرقام والأوصاف وطريقة التحضير وأي شيء بين أقواس"""
        if not text:
            return ''

        # احذف محتوى الأقواس ( ) [ ] { }
        text = re.sub(r'[\(\[\{].*?[\)\]\}]', ' ', text)

        # أرقام عربية/إنجليزية + الكسور الشائعة ½ ¼ ¾
        text = re.sub(r'[0-9٠-٩]+([\/\.][0-9٠-٩]+)?', ' ', text)
        text = re.sub(r'[¼½¾⅓⅔⅛⅜⅝⅞]', ' ', text)

        # تطبيع
        text_norm = self.normalize_ar(text)

        # أنماط للكلمات الواجب حذفها
        quantity_pattern = r'\b(?:' + '|'.join(re.escape(self.normalize_ar(w)) for w in self.quantity_words) + r')\b'
        method_pattern   = r'\b(?:' + '|'.join(re.escape(self.normalize_ar(w)) for w in self.method_words)   + r')\b'
        adj_pattern      = r'\b(?:' + '|'.join(re.escape(self.normalize_ar(w)) for w in self.adjective_words)+ r')\b'

        # حذف الكميات/الأفعال/الأوصاف
        text_norm = re.sub(quantity_pattern, ' ', text_norm)
        text_norm = re.sub(method_pattern,   ' ', text_norm)
        text_norm = re.sub(adj_pattern,      ' ', text_norm)

        # تنظيف نهائي
        text_norm = re.sub(r'\s+', ' ', text_norm).strip()
        return text_norm

    def is_ingredient(self, text):
        """يرجع (True, [ingredients]) فقط إذا وُجدت مكوّنات أساسية صريحة"""
        cleaned = self.clean_ingredient_text(text)
        if not cleaned:
            return False, []

        # طبّع الكلمات الأساسية للمقارنة
        ing_keywords_norm = { self.normalize_ar(w) for w in self.ingredient_keywords }

        # قطّع الجملة إلى كلمات وطبعها
        tokens = [self.normalize_ar(t) for t in self.tokenize(cleaned)]

        # تراكيب ثنائية (مثلاً: زيت زيتون)
        joined_tokens = set()
        for i in range(len(tokens)-1):
            bigram = tokens[i] + ' ' + tokens[i+1]
            if bigram in ing_keywords_norm:
                joined_tokens.add(bigram)

        # الكلمات المفردة
        single_hits = set(t for t in tokens if t in ing_keywords_norm)

        hits = sorted(joined_tokens | single_hits)

        if not hits:
            return False, []
        return True, hits

    def filter_to_known_ingredients(self, items):
        """فلترة نهائية للمكونات لتطابق القاموس (تطبيعاً) مع إزالة التكرار"""
        kw = { self.normalize_ar(w) for w in self.ingredient_keywords }
        out = []
        seen = set()
        for it in items:
            itn = self.normalize_ar(it)
            if itn in kw and itn not in seen:
                out.append(it)
                seen.add(itn)
        return out

    # ====== استخراج تفاصيل الطبق ======
    def extract_dish_details(self, dish_url):
        """Extract details from a single dish page"""
        html_content = self.get_page_content(dish_url)
        if not html_content:
            return None

        soup = BeautifulSoup(html_content, 'html.parser')

        # اسم الطبق
        title_element = soup.find('h1') or soup.find('h2') or soup.find('title')
        dish_name = title_element.get_text().strip() if title_element else "Unknown Dish"

        # صورة الطبق
        image_url = None
        meta_image = soup.find('meta', property='og:image')
        if meta_image:
            image_url = meta_image.get('content', '')
        else:
            img_element = soup.find('img', {'src': re.compile(r'\.(jpg|jpeg|png|webp)', re.I)})
            if img_element:
                image_url = img_element.get('src', '')
                if image_url and not image_url.startswith('http'):
                    image_url = f"{self.base_url}{image_url}"

        # استخراج المكونات الأساسية فقط
        collected = []

        # من schema.org
        ingredient_elements = soup.select('[itemprop="recipeIngredient"]')
        if ingredient_elements:
            for el in ingredient_elements:
                text = el.get_text().strip()
                is_ing, hits = self.is_ingredient(text)
                if is_ing and hits:
                    collected.append(hits)

        # حاويات شائعة
        if not collected:
            possible_containers = soup.select('.ingredients, .recipe-ingredients, .mkd-recipe-ingredients')
            for container in possible_containers:
                list_items = container.select('li')
                if list_items:
                    for li in list_items:
                        text = li.get_text().strip()
                        is_ing, hits = self.is_ingredient(text)
                        if is_ing and hits:
                            collected.append(hits)
                    break

        # بالبحث النصي
        if not collected:
            for element in soup.find_all(text=re.compile(r'مقادير|مكونات|المقادير|المكونات', re.IGNORECASE)):
                container = element.find_parent()
                if container:
                    list_items = container.find_all('li')
                    if list_items:
                        for li in list_items:
                            text = li.get_text().strip()
                            is_ing, hits = self.is_ingredient(text)
                            if is_ing and hits:
                                collected.append(hits)
                        break

        # تسطيح + إزالة تكرار + فلترة نهائية
        ingredients = []
        if collected:
            flat = []
            for item in collected:
                if isinstance(item, list):
                    flat.extend(item)
                elif isinstance(item, str):
                    ok, hits = self.is_ingredient(item)
                    if ok:
                        flat.extend(hits)
            # إزالة التكرار مع الحفاظ على الترتيب
            seen = set()
            tmp = []
            for x in flat:
                if x and x not in seen:
                    seen.add(x)
                    tmp.append(x)
            ingredients = self.filter_to_known_ingredients(tmp)

        scrape_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        return {
            'dish_name': dish_name,
            'ingredients': ingredients,
            'image_url': image_url,
            'dish_url': dish_url,
            'scrape_date': scrape_date
        }

    # ====== التشغيل العام ======
    def scrape_saudi_dishes(self, max_pages=5):
        """Main method to scrape Saudi dishes"""
        print("Starting to scrape Saudi dishes...")
        all_dish_links = []

        # تصفح صفحات المطبخ السعودي
        for page in range(1, max_pages + 1):
            page_url = f"{self.saudi_cuisine_url}?page={page}" if page > 1 else self.saudi_cuisine_url
            print(f"Scraping page {page}: {page_url}")

            html_content = self.get_page_content(page_url)
            if not html_content:
                print(f"Failed to retrieve page {page}")
                continue

            dish_links = self.extract_dish_links(html_content)
            all_dish_links.extend(dish_links)

            print(f"Found {len(dish_links)} dishes on page {page}")
            time.sleep(1)  # احترام للموقع

        # إزالة التكرار
        all_dish_links = list(set(all_dish_links))
        print(f"Total unique dishes found: {len(all_dish_links)}")

        # سحب كل صفحة طبق
        for i, dish_url in enumerate(all_dish_links, 1):
            print(f"Scraping dish {i}/{len(all_dish_links)}: {dish_url}")

            dish_data = self.extract_dish_details(dish_url)
            if dish_data and dish_data['ingredients']:  # فقط لو وجدنا مكونات
                self.dishes_data.append(dish_data)

            time.sleep(1)  # احترام للموقع

        print(f"Successfully scraped {len(self.dishes_data)} dishes with ingredients")
        return self.dishes_data

    # ====== الحفظ ======
    def save_to_csv(self, filename="saudi_dishes.csv"):
        """Save scraped data to CSV file"""
        if not self.dishes_data:
            print("No data to save")
            return

        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['dish_name', 'ingredients', 'image_url', 'dish_url', 'scrape_date']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for dish in self.dishes_data:
                dish_copy = dish.copy()
                dish_copy['ingredients'] = '|'.join(dish['ingredients'])
                writer.writerow(dish_copy)

        print(f"Data saved to {filename}")

    def save_to_json(self, filename="saudi_dishes.json"):
        """Save scraped data to JSON file"""
        if not self.dishes_data:
            print("No data to save")
            return

        with open(filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(self.dishes_data, jsonfile, ensure_ascii=False, indent=2)

        print(f"Data saved to {filename}")

# ====== التشغيل المباشر ======
if __name__ == "__main__":
    scraper = SaudiDishScraper()
    dishes = scraper.scrape_saudi_dishes(max_pages=3)  # غيّر عدد الصفحات حسب حاجتك

    if dishes:
        scraper.save_to_csv()
        scraper.save_to_json()

        # طباعة عينة
        print("\nSample of scraped data:")
        for i, dish in enumerate(dishes[:3]):
            print(f"\nDish {i+1}:")
            print(f"Name: {dish['dish_name']}")
            print(f"Ingredients: {dish['ingredients']}")
            print(f"Image URL: {dish['image_url']}")
            print(f"Scraped on: {dish['scrape_date']}")


Starting to scrape Saudi dishes...
Scraping page 1: https://kitchen.sayidaty.net/recipes/index/cuisine/2419
Found 18 dishes on page 1
Scraping page 2: https://kitchen.sayidaty.net/recipes/index/cuisine/2419?page=2
Found 19 dishes on page 2
Scraping page 3: https://kitchen.sayidaty.net/recipes/index/cuisine/2419?page=3
Found 19 dishes on page 3
Total unique dishes found: 55
Scraping dish 1/55: https://kitchen.sayidaty.net/node/35528/لقيمات-محشية-بالقشطة/حلويات/وصفات


/tmp/ipython-input-2987142132.py:401: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for element in soup.find_all(text=re.compile(r'مقادير|مكونات|المقادير|المكونات', re.IGNORECASE)):


Scraping dish 2/55: https://kitchen.sayidaty.net/node/35168/كباب-ميرو-السعودي/وصفات-طبخ/وصفات
Scraping dish 3/55: https://kitchen.sayidaty.net/node/33971/طريقة-عمل-عيش-أبو-اللحم/وصفات-طبخ/وصفات
Scraping dish 4/55: https://kitchen.sayidaty.net/node/33901/كبسة-اللحم-بالأرز-البسمتي/وصفات-طبخ/وصفات
Scraping dish 5/55: https://kitchen.sayidaty.net/node/33617/السمبوسة-السعودية-بالجبنة/المقبلات/وصفات-رمضانية
Scraping dish 6/55: https://kitchen.sayidaty.net/node/33512/اللقيمات-السعودية-التقليدية/حلويات/وصفات-رمضانية
Scraping dish 7/55: https://kitchen.sayidaty.net/node/33537/الكبسة-وأسياخ-الدجاج-المشوية-بتتبيلة-الروب-من-ساديا-بالفيديو/وصفات-طبخ/وصفات-الفيديو/وصفات-رمضانية
Scraping dish 8/55: https://kitchen.sayidaty.net/node/34853/كبسة-اللحمة-السعودية-وسلطة-الدقوس/وصفات-طبخ/وصفات
Scraping dish 9/55: https://kitchen.sayidaty.net/node/34837/المندي-السعودي-في-المنزل/وصفات-طبخ/وصفات
Scraping dish 10/55: https://kitchen.sayidaty.net/node/35403/كبسة-الدجاج-على-الطريقة-السعودية/وصفات-طبخ/وصفات
Scrapi